In [261]:
import pandas as pd
import numpy as np


In [262]:
train = pd.read_csv('clean.csv')

In [263]:
test = pd.read_csv('clean_test.csv')

In [264]:
train.shape,test.shape

((59400, 23), (14850, 22))

In [265]:
#drop this unnamed column 
train.drop('Unnamed: 0', axis=1, inplace=True)
test.drop('Unnamed: 0', axis=1, inplace=True)
target=train.pop('status_group')

In [266]:
train['train']=1
test['train']=0
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 22 columns):
id                   59400 non-null int64
amount_tsh           59400 non-null float64
funder               59400 non-null object
gps_height           59400 non-null float64
installer            59400 non-null object
longitude            59400 non-null float64
latitude             59400 non-null float64
basin                59400 non-null object
region               59400 non-null object
district_code        59400 non-null int64
lga                  59400 non-null object
population           59400 non-null float64
scheme_management    59400 non-null object
extraction_type      59400 non-null object
management           59400 non-null object
payment_type         59400 non-null object
water_quality        59400 non-null object
quantity             59400 non-null object
source               59400 non-null object
waterpoint_type      59400 non-null object
operational_year     59400 n

In [267]:
train.shape,test.shape

((59400, 22), (14850, 22))

In [268]:
combined = pd.concat([train, test])



In [269]:
combined.info()
combined.drop('construction_year',axis=1,inplace=True)
combined.drop('date_recorded',axis=1,inplace=True)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 74250 entries, 0 to 14849
Data columns (total 22 columns):
id                   74250 non-null int64
amount_tsh           74250 non-null float64
funder               74250 non-null object
gps_height           74250 non-null float64
installer            74250 non-null object
longitude            74250 non-null float64
latitude             74250 non-null float64
basin                74250 non-null object
region               74250 non-null object
district_code        74250 non-null int64
lga                  74250 non-null object
population           74250 non-null float64
scheme_management    74250 non-null object
extraction_type      74250 non-null object
management           74250 non-null object
payment_type         74250 non-null object
water_quality        74250 non-null object
quantity             74250 non-null object
source               74250 non-null object
waterpoint_type      74250 non-null object
operational_year     74250 n

In [270]:
combined['funder'] = pd.factorize(combined['funder'])[0]
combined['scheme_management'] = pd.factorize(combined['scheme_management'])[0]
combined['extraction_type'] = pd.factorize(combined['extraction_type'])[0]
combined['management'] = pd.factorize(combined['management'])[0]
combined['payment_type'] = pd.factorize(combined['payment_type'])[0]
combined['water_quality'] = pd.factorize(combined['water_quality'])[0]
combined['quantity'] = pd.factorize(combined['quantity'])[0]
combined['source'] = pd.factorize(combined['source'])[0]
combined['waterpoint_type'] = pd.factorize(combined['waterpoint_type'])[0]
combined['basin'] = pd.factorize(combined['basin'])[0]
combined['region'] = pd.factorize(combined['region'])[0]
combined['lga'] = pd.factorize(combined['lga'])[0]
combined['district_code'] = pd.factorize(combined['district_code'])[0]
combined['operational_year'] = pd.factorize(combined['operational_year'])[0]
combined.district_code.head(5)

In [271]:
train_df = combined[combined["train"] == 1]
test_df = combined[combined["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [272]:
#define X train
X = train_df
#y_train indentified at the beginning 
y = target

In [273]:
from sklearn.ensemble import RandomForestClassifier
model_rfc = RandomForestClassifier(n_estimators=1000)

In [274]:
#from sklearn.svm import SVC
#clf = SVC()

In [275]:
from xgboost import XGBClassifier
model = XGBClassifier(objective = 'multi:softmax', booster = 'gbtree', nrounds = 'min.error.idx', 
                      num_class = 4, maximize = False, eval_metric = 'merror', eta = .2,
                      max_depth = 14, colsample_bytree = .4)

In [276]:
#from sklearn.neighbors import KNeighborsClassifier
#knn=KNeighborsClassifier(n_neighbors=1000)

In [277]:
#from sklearn.ensemble import VotingClassifier
#eclf1 = VotingClassifier(estimators=[('xg', model), ('rf', model_rfc)],weights=[1,2], voting='hard')

In [278]:
from sklearn.cross_validation import cross_val_score
cross_val_score(model_rfc, X, y, cv=3)

array([ 0.80292929,  0.80282828,  0.80035354])

In [279]:
model_rfc.fit(X,y)
X.info()
importances = model_rfc.feature_importances_
importances

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 0 to 59399
Data columns (total 18 columns):
amount_tsh          59400 non-null float64
funder              59400 non-null int64
gps_height          59400 non-null float64
installer           59400 non-null int64
longitude           59400 non-null float64
latitude            59400 non-null float64
basin               59400 non-null int64
region              59400 non-null int64
district_code       59400 non-null int64
lga                 59400 non-null int64
population          59400 non-null float64
extraction_type     59400 non-null int64
management          59400 non-null int64
payment_type        59400 non-null int64
quantity            59400 non-null int64
source              59400 non-null int64
waterpoint_type     59400 non-null int64
operational_year    59400 non-null int64
dtypes: float64(5), int64(13)
memory usage: 8.6 MB


array([ 0.02600663,  0.04767696,  0.07709058,  0.03804899,  0.15242568,
        0.14945111,  0.01635002,  0.02270284,  0.021218  ,  0.03128829,
        0.05169935,  0.05699834,  0.02138889,  0.03299838,  0.11918174,
        0.03078782,  0.05758706,  0.04709931])

In [280]:
importances = model_rfc.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')


Feature ranking:
1. feature 4 (0.152426)
2. feature 5 (0.149451)
3. feature 14 (0.119182)
4. feature 2 (0.077091)
5. feature 16 (0.057587)
6. feature 11 (0.056998)
7. feature 10 (0.051699)
8. feature 1 (0.047677)
9. feature 17 (0.047099)
10. feature 3 (0.038049)
11. feature 13 (0.032998)
12. feature 9 (0.031288)
13. feature 15 (0.030788)
14. feature 0 (0.026007)
15. feature 7 (0.022703)
16. feature 12 (0.021389)
17. feature 8 (0.021218)
18. feature 6 (0.016350)
longitude, latitude, quantity, gps_height, waterpoint_type, extraction_type, population, funder, operational_year, installer, payment_type, lga, source, amount_tsh, region, management, district_code, basin, 

In [ ]:
#ranking of features-
#longitude,latitude,gps_height,ward,funder,operational_year,population,

In [258]:
#for xgboost
'''from sklearn.cross_validation import cross_val_score
print(cross_val_score(model, X, y, cv=3))
model.fit(X,y)
importances = model.feature_importances_
importances
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")

for f in range(X.shape[1]):
    print(X.columns[indices[f]],end=', ')'''

[ 0.81040404  0.81277778  0.80919192]
Feature ranking:
longitude, latitude, gps_height, funder, operational_year, population, installer, amount_tsh, lga, payment_type, district_code, extraction_type, basin, management, source, region, scheme_management, waterpoint_type, water_quality, quantity, 

In [259]:
#for f in range(X.shape[1]):
 #   print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

1. feature 4 (0.207134)
2. feature 5 (0.191001)
3. feature 2 (0.118806)
4. feature 1 (0.067732)
5. feature 19 (0.061888)
6. feature 10 (0.061443)
7. feature 3 (0.049777)
8. feature 0 (0.032798)
9. feature 9 (0.029129)
10. feature 14 (0.022409)
11. feature 8 (0.022304)
12. feature 12 (0.021030)
13. feature 6 (0.018021)
14. feature 13 (0.016567)
15. feature 17 (0.015953)
16. feature 7 (0.015375)
17. feature 11 (0.013728)
18. feature 18 (0.012758)
19. feature 15 (0.011964)
20. feature 16 (0.010183)


In [ ]:
####xg-nrounds=1000-array([ 0.80924242,  0.81217172,  0.81186869])
#xg-max_depth=5-array([ 0.76282828,  0.76823232,  0.76994949])
#xg-max_depth=30-array([ 0.80772727,  0.80782828,  0.805     ])
#model_rfc-array([ 0.80479798,  0.80267677,  0.80207071])
#xg-nrounds=500-array([ 0.80939394,  0.80828283,  0.80777778])
#xg-nrounds=1000-array([ 0.80939394,  0.80828283,  0.80777778])
#xg-max_depth=20-array([ 0.8089899 ,  0.80893939,  0.80848485])
#xg-max_depth=16-array([ 0.80838384,  0.80984848,  0.80742424])
#xg-max_depth=12,nfold=5,array([ 0.80939394,  0.80828283,  0.80777778])
#xg-max_depth=12,nfold=6,array([ 0.80939394,  0.80828283,  0.80777778])
#xg-max_depth=12,nfold=6,nrounds=1200,early_stopping_rounds = 9-

In [233]:
X_test=test_df


In [234]:
model_rfc.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [235]:
a=X_test['id']
X_test.drop(['id'],axis=1, inplace=True)
y_pred = model.predict(X_test)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [236]:
y_pred=pd.DataFrame(y_pred)
y_pred['id']=a
y_pred.columns=['status_group','id']
y_pred=y_pred[['id','status_group']]

In [237]:
y_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 2 columns):
id              14850 non-null int64
status_group    14850 non-null object
dtypes: int64(1), object(1)
memory usage: 232.1+ KB


In [238]:
pd.DataFrame(y_pred).to_csv("submission_xg.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 4 columns):
Unnamed: 0      14850 non-null int64
0               14850 non-null object
id              14850 non-null int64
status_group    14850 non-null object
dtypes: int64(2), object(2)
memory usage: 464.1+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 2 columns):
id              14850 non-null int64
status_group    14850 non-null object
dtypes: int64(1), object(1)
memory usage: 232.1+ KB


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14850 entries, 0 to 14849
Data columns (total 2 columns):
id              14850 non-null int64
status_group    14850 non-null object
dtypes: int64(1), object(1)
memory usage: 232.1+ KB
